# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 28.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [148]:
# Load the libraries as required.

%load_ext dotenv
%dotenv 

import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
import pandas as pd
import numpy as np
import os

# for Get X and Y
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate
# grid search
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score
from sklearn.naive_bayes import GaussianNB

# one hot encoder #3B
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier


# SHAP 
import shap

#file_path = os.getenv("../../05_src/data/credit/cs-training.csv")
#df_raw = pd.read_csv(file_path)


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [101]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [102]:
# 04 Notebook - Cross-validation of simple pipeline

# capture all data features until we would know which ones to use; possibly ref all col then use drop?
# Assuming what will be the predictors and what we are trying to predict
# we are assuming we can predict "area" of the fire by looking at the other predictors

# removing missing datapoints from target variable
fires_dt.dropna(subset= ['area'])

X = fires_dt.drop(columns = "area") # taking all other the predictors except area
y = fires_dt["area"]

In [103]:
# Load data

numeric_cols_X = [
    'coord_x', 'coord_y','ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'
]

cat_cols_X = ['month', 'day']


# spliting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [104]:
# one hot encoding: convert it to represent 0 or 1

#onehot.fit(fires_dt[columns])

# standardize the numeric columns 
preproc1 = ColumnTransformer([
    # standard
    ('numeric_transform_OH', StandardScaler(), numeric_cols_X),
    # uses one hot
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), cat_cols_X  )
    # let the other passthrough
], remainder='passthrough')

preproc1
####
# standard scaling method


ColumnTransformer(remainder='passthrough',
                  transformers=[('numeric_transform_OH', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('onehot',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc2 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [105]:
preproc2 = ColumnTransformer([
        # non linear
    ('transform', PowerTransformer(method='yeo-johnson'), numeric_cols_X),
     # uses one hot
    ('numeric_transform_OH2', StandardScaler(), numeric_cols_X),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), cat_cols_X)
], 
# let the other passthrough
remainder='passthrough')

preproc2

ColumnTransformer(remainder='passthrough',
                  transformers=[('transform', PowerTransformer(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('numeric_transform_OH2', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('onehot',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [106]:
# Pipeline A = preproc1 + baseline

Pipeline_A = Pipeline([
    ('preproc', preproc1), 
    ('clf', Lasso() )
])

Pipeline_A




Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_transform_OH',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('clf', Lasso())])

In [116]:
# Pipeline B = preproc2 + baseline


Pipeline_B = Pipeline([
    ('preproc', preproc2),
    ('clf', Lasso())
])

Pipeline_B




Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('transform',
                                                  PowerTransformer(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('numeric_transform_OH2',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('clf', Lasso())])

In [137]:
# Pipeline C = preproc1 + advanced model

# preprocessing
Pipeline_C = Pipeline([
    ('preprocess', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()) ])),
    ('model', LogisticRegression())
])
Pipeline_C

Pipeline(steps=[('preprocess',
                 Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                 ('scaler', StandardScaler())])),
                ('model', LogisticRegression())])

In [ ]:
# Pipeline D = preproc2 + advanced model

# preprocessing
Pipeline_D = Pipeline([
    ('preprocess', preproc2),
    ('model', LinearRegression())
])
Pipeline_D

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('transform',
                                                  PowerTransformer(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('numeric_transform_OH2',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('model', GaussianNB())])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [110]:
# range of items we can score against

scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'f1': 'f1',
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall'
}

scoring_lasso = {
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'r2': 'r2'
}

In [150]:
# 05_Lab

ctransform_simple_A = ColumnTransformer([
    ('numeric_simple', Pipeline_A, numeric_cols_X ),
], remainder='passthrough')


pipe_A_lasso = Pipeline([
    ('preprocess', preproc1), 
    ('clf', Lasso())
])


# dependent on the scoring you want to use

# Pipeline A
# param grid

param_grid_A = {
    'clf__alpha': [0.01, 0.1, 1.0]
}


# 05_Lab
grid_cv = GridSearchCV(
    estimator=pipe_A_lasso , 
    # CV off of the grid for A
    param_grid=param_grid_A, 
    scoring = scoring_lasso, 
    cv = 5,
    refit = 'neg_mean_squared_error',
    error_score='raise')
grid_cv.fit(X_train, Y_train)

# Results
grid_cv.cv_results_
#print(f"Best score: {grid_cv.best_score_}" )

{'mean_fit_time': array([0.01120205, 0.00740185, 0.00740328]),
 'std_fit_time': array([0.00285732, 0.00049006, 0.00048841]),
 'mean_score_time': array([0.00380135, 0.00400085, 0.00349364]),
 'std_score_time': array([4.00185709e-04, 3.50402318e-07, 4.47213941e-04]),
 'param_clf__alpha': masked_array(data=[0.01, 0.1, 1.0],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__alpha': 0.01}, {'clf__alpha': 0.1}, {'clf__alpha': 1.0}],
 'split0_test_neg_mean_squared_error': array([-1664.06674253, -1660.00541132, -1589.0980909 ]),
 'split1_test_neg_mean_squared_error': array([-379.1410024 , -354.2310884 , -284.19141491]),
 'split2_test_neg_mean_squared_error': array([-1117.89844451, -1105.19976073,  -812.21972556]),
 'split3_test_neg_mean_squared_error': array([-7265.66349416, -7256.70558865, -7181.71533415]),
 'split4_test_neg_mean_squared_error': array([-718.22010598, -680.62210289, -590.03492787]),
 'mean_test_neg_mean_squared_err

In [151]:
param_grid_B = {
    'clf__alpha': [0.01, 0.1, 1.0]
}

grid_search_B = GridSearchCV(
    estimator=Pipeline_B,
    param_grid=param_grid_B,
    scoring='neg_mean_squared_error',
    cv=5,
    refit='neg_mean_squared_error',
    error_score='raise'
)

grid_search_B.fit(X_train, Y_train)

grid_cv.cv_results_

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.343e+04, tolerance: 7.234e+01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.386e+04, tolerance: 8.391e+01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.697e+03, tolerance: 7.991e+01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.342e+04, tolerance: 2.631e+01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.265e+04, tolerance: 8.092e+01


{'mean_fit_time': array([0.01120205, 0.00740185, 0.00740328]),
 'std_fit_time': array([0.00285732, 0.00049006, 0.00048841]),
 'mean_score_time': array([0.00380135, 0.00400085, 0.00349364]),
 'std_score_time': array([4.00185709e-04, 3.50402318e-07, 4.47213941e-04]),
 'param_clf__alpha': masked_array(data=[0.01, 0.1, 1.0],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__alpha': 0.01}, {'clf__alpha': 0.1}, {'clf__alpha': 1.0}],
 'split0_test_neg_mean_squared_error': array([-1664.06674253, -1660.00541132, -1589.0980909 ]),
 'split1_test_neg_mean_squared_error': array([-379.1410024 , -354.2310884 , -284.19141491]),
 'split2_test_neg_mean_squared_error': array([-1117.89844451, -1105.19976073,  -812.21972556]),
 'split3_test_neg_mean_squared_error': array([-7265.66349416, -7256.70558865, -7181.71533415]),
 'split4_test_neg_mean_squared_error': array([-718.22010598, -680.62210289, -590.03492787]),
 'mean_test_neg_mean_squared_err

In [155]:
# pipeline C


ctransform_simple2 = ColumnTransformer([
    ('numeric_transform', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('standardizer', StandardScaler())
    ]), numeric_cols_X),
    
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['month', 'day'])
], remainder='passthrough')


pipe_lr2 = Pipeline([
    ('preprocess', preproc1),
    ('clf', LinearRegression())
])

param_grid_C = {
    'clf__fit_intercept': [True, False]
}


# Grid Search
grid_cv_C = GridSearchCV(
    estimator=pipe_lr2,  
    param_grid=param_grid_C,  
    scoring='neg_log_loss', 
    cv=5, 
    refit='neg_log_loss'
)

# Fit the model
grid_cv_C.fit(X_train, Y_train)

grid_cv_C.cv_results_
#####
""" 
param_grid = {
    'model__C': [0.1, 1, 10, 100],
    'model__solver': ['lbfgs', 'liblinear', 'saga'],
    'model__max_iter': [1,10,100],
    'model__penalty': ['l2']
}

grid_search_C = GridSearchCV(Pipeline_C, param_grid, cv=5, n_jobs=-1, verbose=1)

# Fit the model
grid_search_C.fit(X_train, Y_train)  

print("Best parameters found: ", grid_search_C.best_params_)
print("Best cross-validation score: ", grid_search_C.best_score_) """

Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 316, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\utils\_available_if.py", line 32, in __get__
    if not self.check(obj):
  File "c:\Users\Work\min

' \nparam_grid = {\n    \'model__C\': [0.1, 1, 10, 100],\n    \'model__solver\': [\'lbfgs\', \'liblinear\', \'saga\'],\n    \'model__max_iter\': [1,10,100],\n    \'model__penalty\': [\'l2\']\n}\n\ngrid_search_C = GridSearchCV(Pipeline_C, param_grid, cv=5, n_jobs=-1, verbose=1)\n\n# Fit the model\ngrid_search_C.fit(X_train, Y_train)  \n\nprint("Best parameters found: ", grid_search_C.best_params_)\nprint("Best cross-validation score: ", grid_search_C.best_score_) '

In [154]:
ctransform_simple_D = ColumnTransformer([
    ('numeric_transform', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('standardizer', StandardScaler())
    ]), numeric_cols_X),
    
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['month', 'day'])
], remainder='passthrough')


pipe_lr_D = Pipeline([
    ('preprocess', preproc1),
    ('clf', LinearRegression())
])

param_grid_D = {
    'clf__fit_intercept': [True, False]
}


# Grid Search
grid_cv_D = GridSearchCV(
    estimator=pipe_lr2,  
    param_grid=param_grid_D,  
    scoring='neg_log_loss', 
    cv=5, 
    refit='neg_log_loss'
)

# Fit the model
grid_cv_D.fit(X_train, Y_train)
grid_cv_D.cv_results_
#grid_cv_D.best_params_

Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 316, in _score
    y_pred = method_caller(clf, "predict_proba", X)
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\Work\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\utils\_available_if.py", line 32, in __get__
    if not self.check(obj):
  File "c:\Users\Work\min

{'mean_fit_time': array([0.00872965, 0.01015186]),
 'std_fit_time': array([0.00099884, 0.00124982]),
 'mean_score_time': array([0.0007957 , 0.00080137]),
 'std_score_time': array([0.00039793, 0.00040069]),
 'param_clf__fit_intercept': masked_array(data=[True, False],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__fit_intercept': True}, {'clf__fit_intercept': False}],
 'split0_test_score': array([nan, nan]),
 'split1_test_score': array([nan, nan]),
 'split2_test_score': array([nan, nan]),
 'split3_test_score': array([nan, nan]),
 'split4_test_score': array([nan, nan]),
 'mean_test_score': array([nan, nan]),
 'std_test_score': array([nan, nan]),
 'rank_test_score': array([1, 1])}

# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
# Shap > 06 Lab

# git add 02_activities/assignments/assignment_2.ipynb

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.